In [168]:
import time
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pytz

In [169]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")

# Specify the path to the chromedriver executable
chrome_driver_path = "C:/Users/ddadmin/Desktop/tools/chromedriver-win64/chromedriver.exe"  # chromedriver path

# Set up the WebDriver with Chrome options
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Now you can access the already opened Chrome session
print(driver.title)  # Print the title of the currently opened page

Hodinkee Japan: ホディンキー / 時計メディア(@hodinkeejapan) • Instagram写真と動画


In [170]:
window_handles = driver.window_handles
for handle in window_handles:
    # Switch to the tab
    driver.switch_to.window(handle)
    
    # Check if the current tab is Instagram by inspecting the URL or title
    if "instagram.com" in driver.current_url:
        print("Found Instagram session")
        print(f"Instagram URL: {driver.current_url}")
        break
    else:
        print(f"Other tab URL: {driver.current_url}")

# Now you can interact with the Instagram session
# Example: Print the page title
print("Page title:", driver.title)

Found Instagram session
Instagram URL: https://www.instagram.com/hodinkeejapan/
Page title: Hodinkee Japan: ホディンキー / 時計メディア(@hodinkeejapan) • Instagram写真と動画


In [171]:
# mainDiv = driver.find_element(By.XPATH,"//main[@role='main']")
# print(mainDiv)

<selenium.webdriver.remote.webelement.WebElement (session="08b6d36ef2f998a58286d6c081030449", element="f.E9B9EEF6F7ED992CEC48A161464A49BF.d.EED170165D8A51EDE026304405762BAA.e.99573")>


In [172]:
# post_links = []
# mainChildContainer = mainDiv.find_elements(By.XPATH,"./div") 
# print(len(mainChildContainer))
# postContainer = mainChildContainer[0].find_elements(By.XPATH,"./div")[-1]
# postStyleDiv = postContainer.find_elements(By.XPATH,"./div")[0]
# postRowContainer = postStyleDiv.find_elements(By.XPATH,"./div") 
# print(len(postRowContainer))

# for div_row in postRowContainer:
#     divs = div_row.find_elements(By.XPATH, "div")
#     # print(f"Number of <div> elements in each row: {len(divs)}")
#     for div in divs:
#         a_tags = div.find_elements(By.TAG_NAME, "a")
#         if len(a_tags) > 0:
#             a_tag = a_tags[0]
#             href = a_tag.get_attribute("href")
#             post_links.append(href)
# print(post_links)
# print(len(post_links))


1
4
['https://www.instagram.com/p/DALSUyhveyl/', 'https://www.instagram.com/p/DAM0D6FyQZV/', 'https://www.instagram.com/p/DAIc5-eSp0G/', 'https://www.instagram.com/p/DAGS1XByGCx/', 'https://www.instagram.com/p/DAFxUQZSDTo/', 'https://www.instagram.com/p/DADl7hSyiyj/', 'https://www.instagram.com/p/DAAwn49RmYw/', 'https://www.instagram.com/p/C_-Xsl9ykQZ/', 'https://www.instagram.com/p/C_7GW1jSqg2/', 'https://www.instagram.com/p/C_5WQTCyDWl/', 'https://www.instagram.com/p/C_4HPztTfEM/', 'https://www.instagram.com/reel/C_21B9cSORC/']
12


In [ ]:
def get_dom_posts():
    post_links = []
    mainDiv = driver.find_element(By.XPATH,"//main[@role='main']")
    mainChildContainer = mainDiv.find_elements(By.XPATH,"./div") 
    print(len(mainChildContainer))
    postContainer = mainChildContainer[0].find_elements(By.XPATH,"./div")[-1]
    postStyleDiv = postContainer.find_elements(By.XPATH,"./div")[0]
    postRowContainer = postStyleDiv.find_elements(By.XPATH,"./div") 
    print(len(postRowContainer))

    for div_row in postRowContainer:
        divs = div_row.find_elements(By.XPATH, "div")
        # print(f"Number of <div> elements in each row: {len(divs)}")
        for div in divs:
            a_tags = div.find_elements(By.TAG_NAME, "a")
            if len(a_tags) > 0:
                a_tag = a_tags[0]
                href = a_tag.get_attribute("href")
                post_links.append(href)
    print(post_links)
    print(len(post_links))
    return post_links

In [ ]:
post_info = {}
post_links = get_dom_posts()


In [173]:
def parseNumber(strNumber):
    if '万' in strNumber:
        str_num = strNumber[0:-1]
        value = float(str_num)*10000
        return str(value)
    return strNumber

In [174]:
def parse_view(div_view_container,count_sperator):
    json_view = {}

    view_total = div_view_container.find_element(By.XPATH,"./div/div/div[2]/div/span")
    json_view['ビュー'] = parseNumber(view_total.text)

    per_follower = div_view_container.find_element(By.XPATH,"./div/div/div[3]/div[2]")
    json_view['フォロワー'] = per_follower.text

    per_unfollower = div_view_container.find_element(By.XPATH,"./div/div/div[3]/div[4]")
    json_view['フォロワー以外'] = per_unfollower.text

    if count_sperator == 7:
        view_source_div = div_view_container.find_element(By.XPATH,"./div/div/div[5]")
        view_source_contents = view_source_div.find_elements(By.XPATH,"./div")
        for content in view_source_contents:
            key_div = content.find_element(By.XPATH,"./div/div[1]/span")
            value_div = content.find_element(By.XPATH,"./div/div[2]/span")
            json_view[key_div.text] = value_div.text

    reach_count_div = div_view_container.find_element(By.XPATH,f"./div/div/div[{count_sperator}]")
    reach_key = reach_count_div.find_element(By.XPATH,"./span")
    reach_value = reach_count_div.find_element(By.XPATH,"./div/span")
    json_view[reach_key.text] = reach_value.text
    return json_view

In [175]:
def parse_interaction(div_interaction_container):
    json_interaction = {}
    div_children = div_interaction_container.find_elements(By.XPATH,'./div/div')
    interaction_container = div_children[1]
    post_interaction_container = div_children[3]
    action_account_container = div_children[5]

    # interaction
    interaction_key = interaction_container.find_element(By.XPATH,'./div/div[1]/span')
    interaction_value = interaction_container.find_element(By.XPATH,'./div/div[1]/div/span')
    json_interaction[interaction_key.text] = interaction_value.text
    # print(f'interaction key = {interaction_key.text}  interaction value = {interaction_value.text}')
    
    follower_key = interaction_container.find_element(By.XPATH,'./div/div[2]/div[1]/span')
    follower_value = interaction_container.find_element(By.XPATH,'./div/div[2]/div[2]')
    json_interaction[follower_key.text] = follower_value.text
    # print(f'follower key = {follower_key.text}  follower value = {follower_value.text}')
    
    unfollower_key = interaction_container.find_element(By.XPATH,'./div/div[2]/div[3]/span')
    unfollower_value = interaction_container.find_element(By.XPATH,'./div/div[2]/div[4]')
    json_interaction[unfollower_key.text] = unfollower_value.text
    # print(f'unfollower key = {unfollower_key.text}  unfollower value = {unfollower_value.text}')

    # post interaction
    post_children = post_interaction_container.find_elements(By.XPATH,'./div/div')
    post_interaction_div = post_children[0]
    post_interaction_key = post_interaction_div.find_element(By.XPATH,'./span')
    post_interaction_value = post_interaction_div.find_element(By.XPATH,'./div/span')
    json_interaction[post_interaction_key.text] = post_interaction_value.text
    # post_interaction_key = post_interaction_container.find_element(By.XPATH,'./div/div[1]/span')
    # post_interaction_value = post_interaction_container.find_element(By.XPATH,'./div/div[1]/div/span')
    # print(f'post_interaction key = {post_interaction_key.text}  post_interaction value = {post_interaction_value.text}')
    for element in post_children[1:]:
        key_div = element.find_element(By.XPATH,'./div/div[1]/span')
        value_div = element.find_element(By.XPATH,'./div/div[2]/span')
        json_interaction[key_div.text] = value_div.text
        # print(f'element key = {key_div.text}   element value = {value_div.text}')
    
    # action account
    action_account_key = action_account_container.find_element(By.XPATH,'./span')
    action_account_value = action_account_container.find_element(By.XPATH,'./div/span')
    # print(f'action account  key = {action_account_key.text}   value = {action_account_value.text}')
    json_interaction[action_account_key.text] = action_account_value.text

    return json_interaction
    

In [176]:
def parse_profile(div_profile_container):
    json_profile = {}
    profile_container = div_profile_container.find_element(By.XPATH,'./div/div[2]/div')
    children_elements = profile_container.find_elements(By.XPATH,'./div')
    profile_activity = children_elements[0]
    profile_activity_key = profile_activity.find_element(By.XPATH,'./span')
    profile_activity_value = profile_activity.find_element(By.XPATH,'./div/span')
    json_profile[profile_activity_key.text] = profile_activity_value.text
    for element in children_elements[1:]:
        key_div = element.find_element(By.XPATH,'./div[1]/span')
        value_div = element.find_element(By.XPATH,'./div[2]/span')
        json_profile[key_div.text] = value_div.text
    return json_profile

In [177]:
async def open_post_href(link):
    json_output = []
    home_window = driver.current_window_handle
    driver.switch_to.new_window('tab')
    driver.get(link)
    #
    time_tag = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.TAG_NAME, "time"))
    )
    date_time = time_tag.get_attribute("datetime")
    print(date_time)

    try:
        insight_button = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,"//div[@role='button' and text()='インサイトを見る']")))
        print(insight_button.text)
        print(insight_button.is_enabled())
        driver.execute_script("arguments[0].click();",insight_button)
        WebDriverWait(driver, 10).until(EC.url_contains('insights'))
                
        # find seperators
        hr_separators = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH,"//hr"))
        )
        count_sperator = len(hr_separators)
        if count_sperator == 7:
            main_separators = [hr_separators[2],hr_separators[5],hr_separators[6]]
        else:
            main_separators = [hr_separators[1],hr_separators[4],hr_separators[5]]
        div_separators = [hr.find_element(By.XPATH,"..") for hr in main_separators]
        div_info_root = div_separators[0].find_element(By.XPATH,"..")
        div_info_children = div_info_root.find_elements(By.XPATH,"./div")
        div_view_container = div_info_children[0]
        div_interaction_container = div_info_children[2]
        div_profile_container = div_info_children[4]


        json_output.append(parse_view(div_view_container,count_sperator))  
        json_output.append(parse_interaction(div_interaction_container)) 
        json_output.append(parse_profile(div_profile_container)) 
        
        print(json_output)
        driver.close()
        driver.switch_to.window(home_window)
        return json_output
    except TimeoutException:
        driver.close()
        driver.switch_to.window(home_window)
        return None
    

In [178]:
test = 0
for link in post_links:
    if test > 3:
        break
    print(f"href == {link}")
    test = test + 1
    insight_data = await open_post_href(link)

href == https://www.instagram.com/p/DALSUyhveyl/
url -- https://www.instagram.com/p/DALSUyhveyl/
2024-09-21T10:46:29.000Z
インサイトを見る
True
insight url -- https://www.instagram.com/insights/media/3461941332044278949/
ビュー = 11000.0
per_follower total フォロワー = 81%
per_unfollower total フォロワー以外 = 19%
profile length - 3
profile_activity key = プロフィールのアクティビティ   profile_activity value = 45
element key = プロフィールへのアクセス   element value = 44
element key = フォロー数   element value = 1
[{'ビュー': '11000.0', 'フォロワー': '81%', 'フォロワー以外': '19%', 'ホーム': '7026', '発見から': '1806', 'その他': '121', 'リーチしたアカウント数': '6783'}, {'インタラクション': '376', 'フォロワー': '67.7%', 'フォロワー以外': '32.3%', '投稿でのインタラクション': '0', '「いいね！」': '339', '保存数': '32', 'コメント': '2', 'シェア数': '0', 'アクションを実行したアカウント': '349'}, {'プロフィールのアクティビティ': '45', 'プロフィールへのアクセス': '44', 'フォロー数': '1'}]
href == https://www.instagram.com/p/DAM0D6FyQZV/
url -- https://www.instagram.com/p/DAM0D6FyQZV/
2024-09-21T02:52:37.000Z
インサイトを見る
True
insight url -- https://www.instagram.com/insights

-----------------------------------------------------------------

In [13]:
def request_posts(page_url,start_date,post_targets):
    response = requests.get(page_url)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        posts = data['data']
        # Print or process the data
        # for post in posts:
        #     print(f"ID: {post['id']}")
        #     print(f"Timestamp: {post['timestamp']}")
        last = posts[len(posts)-1]
        last_timestamp_str = last['timestamp']
        last_timestamp = datetime.strptime(last_timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
        print(last_timestamp)
        print(start_date)
        print(last_timestamp > start_date)
        if last_timestamp >= start_date:
            # concate
            post_targets = post_targets + posts
            # request next page
            paging = data['paging']
            next_page_url = paging['next']
            request_posts(next_page_url,start_date,post_targets)
        else:
            for post in posts:
                post_timestamp_str = post['timestamp']
                post_timestamp = datetime.strptime(post_timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
                if post_timestamp > start_date:
                   post_targets.append(post) 
            return post_targets
    else:
        # Print the error if the request was not successful
        print(f"Error: {response.status_code}")
        print(response.json())

In [15]:
ACCESS_TOKEN = 'IGQWRNTExZAcEZAzV3FuU0g3NWJKMnhzRW9tZAkV4eUtHZA1JYZA0hWaEp6Y183MmJneHNSUEpSeklVQXBBQktzcnl5aEtPUDVSU0pwUjA2a3ZAqQnFmMjdRbGFPR3YzM0lyd3Q3ZAHo2b01CY1JhZA1lkbmxRbjVlVldfZA3cZD'
posts_url = f'https://graph.instagram.com/me/media?fields=id,timestamp&access_token={ACCESS_TOKEN}'
print(posts_url)
start_date_str = '2024-06-01'
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
timezone = pytz.timezone('UTC')
start_date = timezone.localize(start_date)
post_targets = []
list_target = request_posts(posts_url,start_date,post_targets)
print(list_target)


https://graph.instagram.com/me/media?fields=id,timestamp&access_token=IGQWRNTExZAcEZAzV3FuU0g3NWJKMnhzRW9tZAkV4eUtHZA1JYZA0hWaEp6Y183MmJneHNSUEpSeklVQXBBQktzcnl5aEtPUDVSU0pwUjA2a3ZAqQnFmMjdRbGFPR3YzM0lyd3Q3ZAHo2b01CY1JhZA1lkbmxRbjVlVldfZA3cZD
2024-07-30 01:45:55+00:00
2024-06-01 00:00:00+00:00
True
2024-07-05 03:00:00+00:00
2024-06-01 00:00:00+00:00
True
2024-06-14 10:11:16+00:00
2024-06-01 00:00:00+00:00
True
2024-05-20 11:15:48+00:00
2024-06-01 00:00:00+00:00
False
None


In [27]:
date_string = "2024-07-30T01:45:55+0000"
date_string_1 = "2024-07-30"

# Convert to a datetime object
dt_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S%z")
dt_object_1 = datetime.strptime(date_string_1, "%Y-%m-%d")

# Print the datetime object
print(dt_object)
print(dt_object_1)

2024-07-30 01:45:55+00:00
2024-07-30 00:00:00


In [34]:
# Optional argument, if not specified will search path.
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/hodinkeejapan/")

# request facebook api to get post count
post_links = []

article_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.TAG_NAME, "article"))
)


def open_post_href(link):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    time_tag = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.TAG_NAME, "time"))
    )
    date_time = time_tag.get_attribute("datetime")
    print(date_time)
    sections = driver.find_elements(By.TAG_NAME,"section")
    print(len(sections))
    #
    # insight_div = driver.find_element(By.XPATH, "//section//section//div[@role='button' and text()='インサイトを見る']")
    # print(f'insight div == {insight_div}')
    #
    for section in sections:
        childDiv = section.find_element(By.XPATH,"//div[@role='button' and text()='インサイトを見る']")
        print(f'sction - childDiv text:  {childDiv.get_attribute('text')}')
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


print(article_element.get_property("className"))
div_row_elements = article_element.find_elements(By.XPATH, "./div/div/div")
# print(f"Number of Rows of <div> elements found: {len(div_row_elements)}")
for div_row in div_row_elements:
    divs = div_row.find_elements(By.XPATH, "./div")
    # print(f"Number of <div> elements in each row: {len(divs)}")
    for div in divs:
        a_tag = div.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        post_links.append(href)
print(f'post_links  = {post_links}')        
        
test = 0      
# for div in div_elements:
for link in post_links:
    if test > 2:
        break
    print(f"href == {link}")
    open_post_href(link)
    test = test + 1
    # time.sleep(6)

# time.sleep(5) # Let the user actually see something!

# driver.quit()

x1iyjqo2
post_links  = ['https://www.instagram.com/hodinkeejapan/p/C-7c6xuyq4O/', 'https://www.instagram.com/hodinkeejapan/p/C-7ZHA2S8n9/', 'https://www.instagram.com/hodinkeejapan/p/C-5B9HtysaY/', 'https://www.instagram.com/hodinkeejapan/p/C-1rzLdR80X/', 'https://www.instagram.com/hodinkeejapan/p/C-zrz8_Sv5X/', 'https://www.instagram.com/hodinkeejapan/p/C-zXsY5Rg9C/', 'https://www.instagram.com/hodinkeejapan/p/C-xKJCUSA08/', 'https://www.instagram.com/hodinkeejapan/p/C-vBsnVyxFy/', 'https://www.instagram.com/hodinkeejapan/p/C-up18zyPap/', 'https://www.instagram.com/hodinkeejapan/p/C-rQ-ThyDqF/', 'https://www.instagram.com/hodinkeejapan/p/C-rQI1_yhC8/', 'https://www.instagram.com/hodinkeejapan/reel/C-pgcW-SfQT/']
href == https://www.instagram.com/hodinkeejapan/p/C-7c6xuyq4O/
2024-08-21T10:39:47.000Z
4


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//section//section//div[@role='button' and text()='インサイトを見る']"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6E7929642+30946]
	(No symbol) [0x00007FF6E78DE3D9]
	(No symbol) [0x00007FF6E77D6FDA]
	(No symbol) [0x00007FF6E782822C]
	(No symbol) [0x00007FF6E782850C]
	(No symbol) [0x00007FF6E786DCB7]
	(No symbol) [0x00007FF6E784CAAF]
	(No symbol) [0x00007FF6E786B041]
	(No symbol) [0x00007FF6E784C813]
	(No symbol) [0x00007FF6E781A6E5]
	(No symbol) [0x00007FF6E781B021]
	GetHandleVerifier [0x00007FF6E7A5F84D+1301229]
	GetHandleVerifier [0x00007FF6E7A6BDC7+1351783]
	GetHandleVerifier [0x00007FF6E7A62A13+1313971]
	GetHandleVerifier [0x00007FF6E795DD16+245686]
	(No symbol) [0x00007FF6E78E759F]
	(No symbol) [0x00007